<img src="../Imagenes/1.jpg">

# Proyecto Pandas - Ataque de tiburones

## Origen y descripción de los datos

El .csv a importar aporta datos sobre ataques de tiburones recopilados por <i>Global Shark Attack File</i>. 

Para más información sobre la base de datos consultar <a href="http://www.sharkattackfile.net/">aquí</a>.

## Preparación de los datos

<img src="../Imagenes/p1.jpg">

### Librerías

Importamos las librerías necesarias para el desarrollo de la tarea.

In [ ]:
import pandas as pd
import numpy as np
import re
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual

### Exploración y primera toma de contacto con los datos

* Importamos el dataset y lo renombramos.

In [ ]:
shark_attacks = pd.read_csv("../Datos/GSAF5.csv", engine = "python") 

* Este comando nos mostrará el tamaño total de nuestra base de datos.

In [ ]:
shark_attacks.shape

* Vemos el nombre de las columnas de nuestro <i>dataset</i>.

In [ ]:
shark_attacks.columns

Podemos empezar a observar que los nombres de las columnas no están normalizados. Posibles cosas a arreglar serían las minúsculas/mayúsculas, borrar los espacios que se ven al final y estandarizar la puntuación por '_'.

* Vemos los tipos de datos que hay en las diferentes columnas.

In [ ]:
shark_attacks.dtypes

* Hacemos una primera visualización de los datos

In [ ]:
shark_attacks.head()

In [ ]:
shark_attacks.tail()

Comparando los primeros registros con los últimos y podemos observar como <CODE>'Case Number'</CODE> y <CODE>Date</CODE> presentan formatos diferentes, <CODE>'Year'</CODE> tiene valores a 0, en <CODE>Name</CODE> hay elementos que deberían ir en <CODE>Sex</CODE>, etc. Pasaremos a observar los atributos que nos interesen con más detalle más adelante.

Las columnas <CODE>'name'</CODE>, <CODE>'fatal_(y/n)'</CODE>, <CODE>'time'</CODE>, <CODE>'investigator_or_source'</CODE>, <CODE>'pdf'</CODE>, <CODE>'href_formula'</CODE>, <CODE>'href'</CODE>,<CODE>'case_number_1'</CODE>,<CODE>'case_number_2'</CODE>, <CODE>'original_order'</CODE> no aportan información relevante para analizar.

* Con esto vemos la cantidad de <CODE>'NaN'</CODE> con los que nos encontramos.

In [ ]:
shark_attacks.isnull().sum()

Primeras observaciones en relación a los <CODE>NaN</CODE>:


* Vemos que las columnas <CODE>'unnamed_22'</CODE> y <CODE>'unnamed_23'</CODE> tienen casi todos los valores <CODE>NaN</CODE>.


* La columna <CODE>'Time'</CODE> tiene más de la mitad de los registros como <CODE>NaN</CODE>. 


* Las columnas <CODE>'Age'</CODE> y <CODE>'Species'</CODE> se acercan a la mitad de registros como <CODE>NaN</CODE>.


## Limpieza de datos

La <b>limpieza de datos </b> es el proceso que sufren los datos en los que se eliminan los que sean erróneos, redundantes o incompletos.

### Normalización de los nombres de los atributos

In [ ]:
def clean (words):
    
    """
    ****************************************************
    Función para limpiar la información,
    pasar la primera a mayúculas y el resto a minúscula,
    eliminaremos espacios innecesarios, el resto serán
    sustituidos por '_', igual que otros elementos de
    puntuación?
    
    Input : lista
    Output : lista
    ****************************************************
    
    """
    c = []
    for word in words:
        word = str(word).lower()
        word = word.strip()
        word = word.replace(".", ' ')
        word = word.replace(" ", '_')
        word = word.replace(":", '_')
        word = word.replace("__", '_')
        word = word.replace ("/", "")
        word = word.replace ("?", "")
        word = word.replace ("(", "")
        word = word.replace (")", "")
        
        c.append (word)
    return c

In [ ]:
def change_names (list1, list2):
    
    """
    *****************************************
    Convierte a diccionario dos listas dadas
    *****************************************
    """
    
    dictchange = dict(zip(list(list1), list(list2)))
    return dictchange

In [ ]:
#Renombramos las columnas.
shark_attacks = shark_attacks.rename(columns = change_names(shark_attacks.columns, clean(shark_attacks.columns)))

In [ ]:
shark_attacks.columns

### Normalización y transformación de datos 

En este apartado nos centraremos en normalizar algunas de las columnas que pueden interesarnos. 

#### Year

Vemos como los datos referentes a la fecha del ataque del tiburon no presentan el mismo formato, intentaremos extraer los años de forma estandarizada para facilitar su posterior análisis.

In [ ]:
shark_attacks['year'].value_counts()

In [ ]:
"""
**************************************************
En los casos en los que el campo 'year' está a 0, 
buscamos si en 'date' está el año.
Si se encuentra con dos fechas hace una media.
Si no encuentra información en ninguna de las columnas
añadirá la media.
**************************************************
"""

year_lst = []
year_lst1 = list(shark_attacks['year'])
        
for row in shark_attacks['date']:
    temp_row = ''.join(re.findall('\-*\d{4}',row))
    temp_row = re.sub('\-','',temp_row)

    if len(str(temp_row))>4:
        temp_row = int((int(temp_row[0:4]) + int(temp_row [4:8]))/2)
        year_lst.append(temp_row)
    else:
        year_lst.append(temp_row)

normalized_years = []
e = 0       
for year in year_lst1:
    try:
        if year != 0:
            normalized_years.append(int(year))
            e+=1
        elif year == 0:
            normalized_years.append(int(year_lst[e]))
            e+=1
    except ValueError:
        normalized_years.append((int(np.mean(normalized_years))))
        
        
shark_attacks['year'] = normalized_years

#### Type

El atributo <CODE>'type' </CODE> está bastante limpio, solo cambiaremos <CODE>'Boating'</CODE> por <CODE>'Boat</CODE>.

In [ ]:
shark_attacks['type'].value_counts()

In [ ]:
shark_attacks['type_attack'] = [re.sub("Boat", "Boating",re.sub("ing", "", i)) for i in shark_attacks['type']]

In [ ]:
shark_attacks['type_attack'].value_counts()

#### Atributos del accidentado

##### Sex

Analizamos los valores que nos encontramos en <CODE>'sex'</CODE> para proceder a su normalización. Veremos cuantos valores hay de cada una de las opciones. También tendremos en cuenta los <b>567</b> valores <CODE>'NaN'</CODE> que vimos durante el proceso de exploración.

In [ ]:
shark_attacks['sex'].value_counts()

Ejecutar solo una de las dos opciones:

###### Opción 1:

Para normalizar el resto de elementos, modificaremos las observaciones en función del porcentaje de cada sexo.

In [ ]:
sex_list=[]
for sex in shark_attacks['sex']:
    
    """
    ******************************************************************
    Con este primer paso inclumos el mayor número de valores correctos
    eliminando los posibles espacios que encontremos.
    ******************************************************************
    """
    
    sex = str(sex)
    if sex.strip() == "M" or sex.strip() == "F":
        sex_list.append(sex.strip())

En esta opción nos centraremos en:

1- sacar % de <CODE>'M'</CODE> y de <CODE>'F'</CODE>,                                         
2- buscar la forma de repartir ese % en el resto de observaciones.

In [ ]:
# % de 'm' y de 'f'
c=round(sex_list.count("F")/len(sex_list)*int(len(shark_attacks['sex'])-len(sex_list)))
d=round(sex_list.count("M")/len(sex_list)*int(len(shark_attacks['sex'])-len(sex_list)))

# creamos una nueva lista con los elementos faltantes en proporción a los % que hemos obtenido antes.
list1 = ("F "* c).split()
list2 = ("M "* d).split()
listrest = list1 + list2

#añadimos la lista a los valores que ya tenemos para poder cambiar luego los valores.
normalized_sex = []
e=0
for sex in shark_attacks['sex']:
    sex= str(sex)
    sex = str(sex.strip())
    if sex == "M" or sex == "F":
        normalized_sex.append(sex)
    else:
        normalized_sex.append(listrest[e])
        e+=1

In [ ]:
temp_normalized_sex = []
for sex in normalized_sex:
    #Cambiamos los valores a :
    if sex == "M":
        sex = "Male"
        temp_normalized_sex.append(sex)
    elif sex == "F":
        sex = "Female"
        temp_normalized_sex.append(sex)
normalized_sex = temp_normalized_sex

In [ ]:
#Cambiamos los valores de la columna
shark_attacks['sex'] = normalized_sex

In [ ]:
shark_attacks['sex'].value_counts()

###### Opción 2:

En esta opción sustituiremos los casos desconocidos por <CODE>'unknown'</CODE>

In [ ]:
normalized_sex = []
for sex in shark_attacks['sex']:
    sex = str(sex)
    if sex.strip() == "M" or sex.strip() == "F":
        normalized_sex.append(sex.strip())
    else:
        normalized_sex.append('Unknown')

In [ ]:
temp_normalized_sex = []
for sex in normalized_sex:
    #Cambiamos los valores a :
    if sex == "M":
        sex = "Male"
        temp_normalized_sex.append(sex)
    elif sex == "F":
        sex = "Female"
        temp_normalized_sex.append(sex)
    else:
        temp_normalized_sex.append(sex)
normalized_sex = temp_normalized_sex

In [ ]:
#Cambiamos los valores de la columna
shark_attacks['sex'] = normalized_sex

Comprobamos que no tenemos <CODE>'NaN'</CODE>

In [ ]:
shark_attacks['sex'].value_counts()

##### Age

Para normalizar <CODE>'age'</CODE> analizaremos primero el tipo de valores que nos encontramos y la frecuencia con la que aparecen.

In [ ]:
pd.unique(shark_attacks['age'])

In [ ]:
shark_attacks['age'].value_counts()

In [ ]:
shark_attacks['age']= clean(shark_attacks['age'])
pd.unique(shark_attacks['age'])

Vemos que los datos han sido tratados de forma muy diferente. Intentaremos agruparlos por categorías en el apartado de discretización.

##### Country

Pasaremos el filtro de limpieza para normalizar los nombres de los paises.

In [ ]:
pd.unique(shark_attacks['country'])

In [ ]:
shark_attacks['country'].value_counts()

In [ ]:
#Limpiaremos los nombres
shark_attacks['country'] = [(str(i).title()).replace("_"," ")
                                           for i in clean(shark_attacks['country'])]

Nos pasa algo parecido que con las edades, los datos han sido tratados de forma diferente, no todos los valores son países. Podría ser interesante hacer una discretización por continentes.

#### Otros atributos

In [ ]:
def nan_to_unknown (column):
    
    """
    ******************************
    Convierte los NaN en 'unknown'
    ******************************
    """
    
    w = column.fillna('Unknown')
    return w

In [ ]:
shark_attacks['species'] = nan_to_unknown(shark_attacks['species'])

In [ ]:
shark_attacks['area'] = nan_to_unknown(shark_attacks['area'])

In [ ]:
shark_attacks['location'] = nan_to_unknown(shark_attacks['location'])

In [ ]:
shark_attacks['activity'] = nan_to_unknown(shark_attacks['activity'])

### Discretización 

La <b>discretización</b> nos permite establecer un creterio por el cual podemos dividir los valores de un atributo en dos o más conjuntos.

#### Ages

En este punto intentaremos agrupar la edad de las víctimas por rangos:
* <CODE>'Newborn'</CODE>: agrupará valores de 0 a 1 año,
* <CODE>'Children'</CODE>: agrupará valores de 1 a 10 años,
* <CODE>'Teen'</CODE>: agrupará valores de 10 a 19 años
* <CODE>'Young'</CODE>: agrupará valores de los 20 a los 30 años,
* <CODE>'Adult'</CODE>: agrupará valores de los 31 a los 55 años,
* <CODE>'Senior'</CODE>: agrupará valores de los 56 a los 75 años,
* <CODE>'Old'</CODE>: agrupará valores desde los 75 años.

In [ ]:
age_list = []
ages_list = ['newborn', 'children','teen','young','adult','senior','old']
        
for row in shark_attacks['age']: 
    
    get_row = "".join(re.findall('(months)',str(row)))
    
    if row.lower() in ages_list:
        age_list.append(row.capitalize()) 

    else:
        
        age_row = "".join(re.findall('\d{1,2}',str(row)))
        age_row = re.sub('\?','',age_row)
        age_row = re.sub('\&','',age_row)
        
        try:
            if get_row == "months":
                age_list.append('Newborn')
            elif int(age_row) >= 1 and int(age_row) <= 10:
                age_list.append('Children')
            elif int(age_row)>10 and int(age_row)<20:
                age_list.append('Teen')
            elif int(age_row)>=20 and int(age_row)<=30:
                age_list.append('Young')
            elif int(age_row)>30 and int(age_row)<=55:
                age_list.append('Adult')
            elif int(age_row)>55 and int(age_row)<75:
                age_list.append('Senior')
            elif int(age_row)>=75:
                age_list.append('Old')

        except ValueError:
            age_list.append('Unknown')

In [ ]:
shark_attacks['age'] = age_list

#### Year to Century

Crearemos una columna nueva para agrupar los años por siglos. Crearemos una columna nueva con esta información a la que llamaremos <CODE>'century</CODE>.

Esto nos será útil para su posterior visualización.

In [ ]:
centuries = []
for year in shark_attacks['year']:
    
    if year <= 100:
        centuries.append('I')
    if year <= 200 and year > 100:
        centuries.append('II')
    if year <= 300 and year > 200:
        centuries.append('III')
    if year <= 400 and year > 300:
        centuries.append('IV')
    if year <= 500 and year > 400:
        centuries.append('V')
    if year <= 600 and year > 500:
        centuries.append('VI')
    if year <= 700 and year > 600:
        centuries.append('VII')
    if year <= 800 and year > 700:
        centuries.append('VIII')
    if year <= 900 and year > 800:
        centuries.append('IX')
    if year <= 1000 and year > 900:
        centuries.append('X')
    if year <= 1100 and year > 1000:
        centuries.append('XI')
    if year <= 1200 and year > 1100:
        centuries.append('XII')
    if year <= 1300 and year > 1200:
        centuries.append('XIII')
    if year <= 1400 and year > 1300:
        centuries.append('XIV')
    if year <= 1500 and year > 1400:
        centuries.append('XV')
    if year <= 1600 and year > 1500:
        centuries.append('XVI')
    if year <= 1700 and year > 1600:
        centuries.append('XVII')
    if year <= 1800 and year > 1700:
        centuries.append('XVIII')
    if year <= 1900 and year > 1800:
        centuries.append('XIX')
    if year <= 2000 and year > 1900:
        centuries.append('XX')
    if year <= 2100 and year > 2000:
        centuries.append('XXI')
    
shark_attacks['century'] = centuries

### Reducción de la dimensionalidad

En este último paso antes de empezar con el análisis se busca trabajar con menos datos y obtener los mismos resultados. Para ello pasaremos a eliminar atributos que no nos aportan información relevante.


Descartamos las columnas <CODE>'date'</CODE>, <CODE>'fatal_(y/n)'</CODE>, <CODE>'time'</CODE>, <CODE>'investigator_or_source'</CODE>, <CODE>'pdf'</CODE>, <CODE>'href_formula'</CODE>, <CODE>'href'</CODE>,<CODE>'case_number_1'</CODE>,<CODE>'case_number_2'</CODE>, <CODE>'original_order'</CODE>, <CODE>'unnamed_22'</CODE> y <CODE>'unnamed_23'</CODE>.

La columna <CODE>'name'</CODE> también será eliminada por la RGPD.

Y pasaremos a ordenar las columnas.

In [ ]:
shark_attacks = shark_attacks.drop(['date','name','type', 'injury', 'fatal_yn', 'time',
                                    'investigator_or_source', 'pdf', 'href_formula', 'href',
                                    'case_number_1', 'case_number_2', 'original_order', 'unnamed_22',
                                    'unnamed_23'], axis=1)

In [ ]:
#Reordenamos las columnas:

column_order = ['case_number','year','century','sex','age',
                'country','area','location', 'activity',
                'type_attack', 'species']

shark_attacks = shark_attacks[column_order]

In [ ]:
shark_attacks.isnull().sum()

Buscaremos las observaciones duplicadas y las eliminaremos.

In [ ]:
shark_attacks = shark_attacks.drop_duplicates()
shark_attacks.shape

Guardamos nuestros datos en un CSV.

In [ ]:
shark_attacks.to_csv("../Datos/shark_attacks.csv", index = False)

## [BONUS] Visualización de resultados

Como hemos visto un poco por encima los witgets intentaremos incluirlos para visualizar los datos.

In [ ]:
ALL = 'All'
def unique_values_plus_ALL(array):
    
    """
    ********************************
    Esta función saca los valores
    únicos de una columna y le añade
    All.
    
    Input = array
    Output = valores únicos del array
    + All.
    ********************************
    """
    
    unique = array.unique().tolist()
    unique.insert(0, ALL)
    return unique

Crearemos un primer desplegable que saque las estadísticas por columna.

In [ ]:
@interact
# El @interact es necesario para que funcione.
def describe(column=list(shark_attacks.columns)):
    
    """
    ********************************************
    Con esta función creamos un desplegable, al 
    seleccionar una de las columnas nos devolverá
    los datos agregados
    *********************************************
    """
    
    print(shark_attacks[column].describe())

Con las siguientes líneas de código conseguimos que nos devuelva los datos según el filtro que le indiquemos.

In [ ]:
lista_century = unique_values_plus_ALL(shark_attacks.century)
lista_age = unique_values_plus_ALL(shark_attacks.age)
lista_sex = unique_values_plus_ALL(shark_attacks.sex)
lista_type_attack = unique_values_plus_ALL(shark_attacks.type_attack)

In [ ]:
#funcion
def filtro(century, age, sex, type_attack):
    
    """
    **************************
    Aplica el filtro a las 3
    columnas que nos interesan.
    **************************
    
    """
    
    if (century == ALL) & (age == ALL) & (sex == ALL)& (type_attack == ALL):
        filtrado = shark_attacks
        
    elif (century == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['age'] == age)& 
                             (shark_attacks['sex'] == sex) & (shark_attacks['type_attack'] == type_attack)]
    elif (age == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['century'] == age)& 
                             (shark_attacks['sex'] == sex)&(shark_attacks['type_attack'] == type_attack)]
    elif (sex == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['century'] == age)& 
                             (shark_attacks['age'] == sex)&(shark_attacks['type_attack'] == type_attack)]
    elif (type_attack == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['century'] == age)& 
                             (shark_attacks['age'] == sex)&(shark_attacks['sex'] == sex)]
    else:
        filtrado = shark_attacks.loc[(shark_attacks['century'] == century) &
                             (shark_attacks['age'] == age) & 
                             (shark_attacks['sex'] == sex) & (shark_attacks['type_attack'] == type_attack)]
        
    return filtrado

Filtro final:

In [ ]:
interact (filtro, 
          century = lista_century,
          age = lista_age,
          sex = lista_sex, 
          type_attack = lista_type_attack);